In [1]:
from resevit_road import restevit_road_cls,restevit_road_cls_lightweight
from torchsummary import summary
import torch
from fvcore.nn import FlopCountAnalysis,flop_count_table
from Load_data import Torch_load
from torch import nn
from tqdm import tqdm
from torch import optim
from Report_model.Report_Torch import Class_Report

In [2]:
model=restevit_road_cls_lightweight(num_class=4)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model= model.to(device)
summary(model, (3,224,224), batch_size=1)
inputs= torch.rand(1,3, 224,224)
inputs=inputs.to(device)
flops=FlopCountAnalysis(model,inputs)
flops.total()
flop_count_table(flops)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 32, 112, 112]           4,704
       BatchNorm2d-2          [1, 32, 112, 112]              64
              ReLU-3          [1, 32, 112, 112]               0
         MaxPool2d-4            [1, 32, 56, 56]               0
         resnet_in-5            [1, 32, 56, 56]               0
            Conv2d-6          [1, 16, 112, 112]             432
       BatchNorm2d-7          [1, 16, 112, 112]              32
         Hardswish-8          [1, 16, 112, 112]               0
         ConvLayer-9          [1, 16, 112, 112]               0
           Conv2d-10          [1, 16, 112, 112]             144
      BatchNorm2d-11          [1, 16, 112, 112]              32
        Hardswish-12          [1, 16, 112, 112]               0
        ConvLayer-13          [1, 16, 112, 112]               0
           Conv2d-14          [1, 16, 1

Unsupported operator aten::add_ encountered 77 time(s)
Unsupported operator aten::max_pool2d encountered 1 time(s)
Unsupported operator aten::hardswish encountered 22 time(s)
Unsupported operator aten::add encountered 47 time(s)
Unsupported operator aten::mul encountered 20 time(s)
Unsupported operator aten::pad encountered 4 time(s)
Unsupported operator aten::div encountered 4 time(s)
Unsupported operator aten::softmax encountered 16 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
base_mdoel.efficientvit_in.input_stem.op_list.1.shortcut, base_mdoel.stage1.stage.op_list.1.shortcut, base_mdoel.stage2.stage.op_list.1.shortcut, base_mdoel.stage3.stage.op_list.1.context_module.shortcut, base_mdoel.stage3.stage.op

'| module                                                  | #parameters or shape   | #flops     |\n|:--------------------------------------------------------|:-----------------------|:-----------|\n| model                                                   | 4.359M                 | 0.69G      |\n|  base_mdoel                                             |  4.357M                |  0.69G     |\n|   base_mdoel.resnet_in                                  |   4.768K               |   61.014M  |\n|    base_mdoel.resnet_in.conv                            |    4.704K              |    59.007M |\n|    base_mdoel.resnet_in.bn                              |    64                  |    2.007M  |\n|   base_mdoel.efficientvit_in.input_stem.op_list         |   0.928K               |   13.447M  |\n|    base_mdoel.efficientvit_in.input_stem.op_list.0      |    0.464K              |    6.423M  |\n|    base_mdoel.efficientvit_in.input_stem.op_list.1.main |    0.464K              |    7.025M  |\n|   base_

In [ ]:
def save_checkpoint(model, filename):
    torch.save(model.state_dict(), filename)
    
output_log="output.txt"
def train(epochs_per_fold=30,batch_size=32,image_size=224):
    for fold,(dataloader_dict) in enumerate(Torch_load(image_size=image_size,batch_size=batch_size,task="multi")()):
        print("Fold:",fold+1)
        criterior= nn.CrossEntropyLoss()
        if fold<1: optimizer = optim.Adam(model.parameters(), lr=0.0001)
        else: optimizer = optim.Adam(model.parameters(), lr=0.00001)
        early_stop_thresh = 5
        best_accuracy = -1
        best_epoch = -1
        stop=False
        for epoch in range(epochs_per_fold):
            if stop==True: break
            with open(output_log, 'a') as f:
                print("Epoch {}/{}".format(epoch, epochs_per_fold),file=f)
            for phase in ["Train", "Val"]:
                if phase == "Train":
                    model.train()
                else:
                    model.eval()
                    
                epoch_loss = 0.0
                epoch_corrects = 0
                
                for inputs, labels in tqdm(dataloader_dict[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    
                    with torch.set_grad_enabled(phase == "Train"):
                        outputs = model(inputs)
                        loss = criterior(outputs, labels)
                        _, preds = torch.max(outputs, 1)
                        
                        if phase == "Train":
                            loss.backward()
                            optimizer.step()
                            
                    epoch_loss += loss.item()*inputs.size(0)
                    epoch_corrects += torch.sum(preds==labels.data)
                epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
                epoch_accuracy = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
                if phase == "Val":
                    if epoch_accuracy > best_accuracy:
                        best_accuracy = epoch_accuracy
                        best_epoch = epoch
                        save_checkpoint(model, "ResEViT_multiclass.pth")
                    elif epoch - best_epoch > early_stop_thresh:
                        print("Early stopped training at epoch %d" % epoch)
                        stop=True  # terminate the training loo
                with open(output_log, 'a') as f:
                    print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_accuracy), file=f)

In [ ]:
train()

In [3]:
checkpoint = torch.load('ResEViT_multiclass.pth')
model.load_state_dict(checkpoint)
Test_data=Torch_load(image_size=224,batch_size=8,phase="Test",task="multi")()
Class_Report(Test_data["Test"],model,device)()

100%|██████████| 52/52 [00:04<00:00, 12.93it/s]

              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       169
         1.0     0.9136    0.9367    0.9250        79
         2.0     0.9505    0.9320    0.9412       103
         3.0     0.9841    0.9841    0.9841        63

    accuracy                         0.9686       414
   macro avg     0.9621    0.9632    0.9626       414
weighted avg     0.9688    0.9686    0.9686       414

